<a href="https://colab.research.google.com/github/SantanaC4/pos_tagging_treebank/blob/main/pos_tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criação de um Part of Speech Tagging com base no Treebank corpus



In [ ]:
import pandas as pd

In [ ]:
url = 'https://raw.githubusercontent.com/SantanaC4/pos_tagging_treebank/main/Sec0-18_training'
df = pd.read_csv(url, header=None, sep='\n')

spliting =  [i.split(" ") for i in df[0]]
extracting_pair = []
for i in spliting:
  if (len(i) == 1):
    extracting_pair.append(tuple(i[0].split("_")))
  else:
    for j in i:
      if (j != ''):
        extracting_pair.append(tuple(j.split("_")))
extracting_pair[0][0]+extracting_pair[0][1]

'PierreNNP'

In [36]:
## Default dictionary 
## Assign default value to key that is not in dictionary
from collections import defaultdict

## Tallying of tag
counts = defaultdict(int)

for (word, tag) in extracting_pair:
  counts[tag] += 1
counts.items()

## Unigram Tagger
pos = defaultdict(list)
for (word, tag) in extracting_pair:
  pos[word].append(tag)

aux = defaultdict(int)
tags_freq = {}
for (word, tags) in pos.items():
  for i in tags:
    aux[i] += 1
  tags_freq[word] = sorted([(value, key) for (key, value) in aux.items()], reverse=True)
  aux = defaultdict(int)

print(tags_freq['offered'])


[(108, 'VBN'), (75, 'VBD'), (4, 'JJ')]
